### E10S Experiment Aurora: Top hang stacks

[Bug 1222972](https://bugzilla.mozilla.org/show_bug.cgi?id=1222972)

This analysis compares e10s and non-e10s hang stacks and their frequencies.

In [1]:
import ujson as json
import matplotlib.pyplot as plt
import pandas as pd
import IPython

from pyspark.sql import SQLContext
from collections import defaultdict
from __future__ import division
from itertools import groupby
from operator import itemgetter

from moztelemetry.histogram import Histogram
from mozaggregator.aggregator import _aggregate_ping, _aggregate_aggregates, _trim_payload, simple_measures_labels, count_histogram_labels

%pylab inline
IPython.core.pylabtools.figsize(16, 7)

Unable to parse whitelist (/home/hadoop/anaconda/lib/python2.7/site-packages/moztelemetry/bucket-whitelist.json). Assuming all histograms are acceptable.
Populating the interactive namespace from numpy and matplotlib


In [2]:
sc.defaultParallelism

16

#### Get e10s and non-e10s partitions

In [4]:
dataset = sqlContext.load("s3://telemetry-parquet/e10s-experiment/generationDate=20151117", "parquet")

How many pings do we have in each branch?

In [5]:
dataset.filter(dataset["experimentBranch"] == "experiment").count()

46749L

In [6]:
dataset.filter(dataset["experimentBranch"] == "control").count()

46716L

Transform Dataframe to RDD of pings

In [7]:
def row_2_ping(row):
    ping = {"payload": {"threadHangStats": json.loads(row.threadHangStats)},
            "e10s": True if row.experimentBranch == "experiment" else False}
    return ping

In [8]:
subset = dataset.rdd.map(row_2_ping)

#### Get top stacks

In [9]:
def get_stacks(subset):
    def yield_ping_stacks(ping):
        for thread in ping["payload"]["threadHangStats"]:
            if thread["name"] != "Gecko":
                continue
            for hang in thread["hangs"]:
                if not hang["stack"]:
                    continue
                values = hang["histogram"]["values"]
                histogram = pd.Series(values.values(), index=map(int, values.keys())).sort_index()
                min_ms = 100
                over_min_ms_count = histogram[histogram.index > min_ms].sum()
                yield (tuple(hang["stack"]), over_min_ms_count)
    return subset.flatMap(yield_ping_stacks).reduceByKey(lambda a, b: a + b).collectAsMap()

In [10]:
we10s_stacks = get_stacks(subset.filter(lambda p: p["e10s"]))

In [11]:
ne10s_stacks = get_stacks(subset.filter(lambda p: not p["e10s"]))

In [12]:
def group_by_top_frame(stacks):
    total_hits = 0
    top_frames = {}
    for stack, hits in stacks.iteritems():
        stack_top_frame = stack[-1]
        if not stack_top_frame in top_frames:
            top_frames[stack_top_frame] = { "frame": stack_top_frame, "stacks": [], "hits": 0 }

        top_frame = top_frames[stack_top_frame]

        # Keep stacks sorted by hits.
        top_frame["stacks"].append((stack, hits))
        top_frame["stacks"].sort(key=lambda d: d[1], reverse=True)

        top_frame["hits"] += hits
        total_hits += hits

    return top_frames, total_hits

def get_stack_hits(stacks, stack):
    for s, h in stacks:
        if s == stack:
            return h
    return 0

In [13]:
we10s_groups, we10s_total_hits = group_by_top_frame(we10s_stacks)

In [14]:
ne10s_groups, ne10s_total_hits = group_by_top_frame(ne10s_stacks)

## Top stacks grouped by top frame

In [15]:
we10s_sorted_groups = sorted(we10s_groups.values(), key=lambda d: d["hits"], reverse=True)

def print_e10s_groups(group_count, stack_count):
    for we10s_group in we10s_sorted_groups[0:group_count]:
        ne10s_group = ne10s_groups.get(we10s_group["frame"], {})
        ne10s_stacks = ne10s_group.get("stacks", [])
        ne10s_hits = ne10s_group.get("hits", 0)

        print "{:.2f}% ({:.2f}%): {} ({})".format(
            100.0 * we10s_group["hits"] / we10s_total_hits,
            100.0 * ne10s_hits / ne10s_total_hits,
            we10s_group["frame"],
            we10s_group["hits"])
        for we10s_stack, we10s_stack_hits in we10s_group["stacks"][0:stack_count]:
            ne10s_stack_hits = get_stack_hits(ne10s_stacks, we10s_stack)
            print "  - {:.4f}% ({:.4f}%):".format(
                100.0 * we10s_stack_hits / we10s_total_hits,
                100.0 * ne10s_stack_hits / ne10s_total_hits)
            print "    {}\n".format("\n    ".join(reversed(we10s_stack)))

#### e10s top frames

The results are in the form: `e10s% (non-e10s%): top frame (total e10s hits)`

In [16]:
print_e10s_groups(25, 0)

43.30% (43.56%): Timer::Fire (44794877)
33.51% (18.44%): Startup::XRE_Main (34669462)
7.34% (0.00%): IPDL::PCookieService::RecvGetCookieString (7598616)
2.04% (3.01%): nsCycleCollector::forgetSkippable (2110463)
1.62% (0.00%): IPDL::PScreenManager::RecvScreenForBrowser (1675449)
1.05% (1.34%): nsViewManager::DispatchEvent (1090817)
1.03% (2.08%): ViewportFrame::BuildDisplayList (1066669)
0.82% (0.15%): nsHttpChannel::OnDataAvailable (847752)
0.76% (0.24%): nsLayoutUtils::GetFramesForArea (783974)
0.49% (3.87%): js::GCRuntime::collect (505067)
0.44% (0.00%): IPDL::PJavaScript::SendGet (450932)
0.42% (0.09%): nsHttpChannel::OnStartRequest (437436)
0.39% (0.00%): IPDL::PStorage::RecvAsyncUpdateItem (398640)
0.34% (0.00%): IPDL::PBrowser::RecvAsyncMessage (348594)
0.28% (0.47%): self-hosted:647 (289674)
0.25% (0.15%): EventDispatcher::Dispatch (254200)
0.23% (1.37%): PresShell::DoReflow (233098)
0.20% (0.00%): IPDL::PNecko::RecvPHttpChannelConstructor (211592)
0.20% (0.03%): nsInputStreamP

#### e10s top stacks of top frames

In [17]:
print_e10s_groups(25, 3)

43.30% (43.56%): Timer::Fire (44794877)
  - 43.2056% (43.2310%):
    Timer::Fire
    Startup::XRE_Main

  - 0.0351% (0.0000%):
    Timer::Fire
    nsViewManager::DispatchEvent
    Startup::XRE_Main

  - 0.0249% (0.0077%):
    Timer::Fire
    services-common/async.js:156
    gre/modules/services-sync/service.js:971
    services-sync/util.js:146
    services-sync/util.js:98
    services-sync/util.js:76
    gre/modules/services-sync/service.js:1270
    services-sync/util.js:76
    gre/modules/services-sync/service.js:1263
    Startup::XRE_Main

33.51% (18.44%): Startup::XRE_Main (34669462)
  - 33.5110% (18.4398%):
    Startup::XRE_Main

7.34% (0.00%): IPDL::PCookieService::RecvGetCookieString (7598616)
  - 7.3378% (0.0041%):
    IPDL::PCookieService::RecvGetCookieString
    Startup::XRE_Main

  - 0.0026% (0.0000%):
    IPDL::PCookieService::RecvGetCookieString
    nsViewManager::DispatchEvent
    Startup::XRE_Main

  - 0.0012% (0.0000%):
    IPDL::PCookieService::RecvGetCookieString
    g

## Top stacks

In [18]:
def print_top_stacks(a_stacks, b_stacks, count):
    from collections import Counter
    a_total_count = sum(a_stacks.values())
    b_total_count = sum(b_stacks.values())
    for a_stack, a_stack_count in Counter(a_stacks).most_common(count):
        b_stack_count = b_stacks.get(a_stack, 0)
        print "- {:.4f}% ({:.4f}%):".format(
            100.0 * a_stack_count / a_total_count,
            100.0 * b_stack_count / b_total_count)
        print "  {}\n".format("\n  ".join(reversed(a_stack)))

#### e10s stacks

In [19]:
print_top_stacks(we10s_stacks, ne10s_stacks, 25)

- 43.2056% (43.2310%):
  Timer::Fire
  Startup::XRE_Main

- 33.5110% (18.4398%):
  Startup::XRE_Main

- 7.3378% (0.0041%):
  IPDL::PCookieService::RecvGetCookieString
  Startup::XRE_Main

- 2.0228% (2.9824%):
  nsCycleCollector::forgetSkippable
  Timer::Fire
  Startup::XRE_Main

- 1.6133% (0.0040%):
  IPDL::PScreenManager::RecvScreenForBrowser
  Startup::XRE_Main

- 0.9656% (1.7687%):
  ViewportFrame::BuildDisplayList
  nsLayoutUtils::GetFramesForArea
  nsLayoutUtils::GetFrameForPoint
  nsViewManager::DispatchEvent
  Timer::Fire
  Startup::XRE_Main

- 0.8883% (1.2465%):
  nsViewManager::DispatchEvent
  Timer::Fire
  Startup::XRE_Main

- 0.8120% (0.1383%):
  nsHttpChannel::OnDataAvailable
  nsInputStreamPump::OnStateTransfer
  nsInputStreamPump::OnInputStreamReady
  Startup::XRE_Main

- 0.6858% (0.2321%):
  nsLayoutUtils::GetFramesForArea
  nsLayoutUtils::GetFrameForPoint
  nsViewManager::DispatchEvent
  Timer::Fire
  Startup::XRE_Main

- 0.4198% (0.0833%):
  nsHttpChannel::OnStartReque

#### non-e10s stacks

In [20]:
print_top_stacks(ne10s_stacks, we10s_stacks, 25)

- 43.2310% (43.2056%):
  Timer::Fire
  Startup::XRE_Main

- 18.4398% (33.5110%):
  Startup::XRE_Main

- 4.7200% (0.0783%):
  (content script)
  Timer::Fire
  Startup::XRE_Main

- 2.9824% (2.0228%):
  nsCycleCollector::forgetSkippable
  Timer::Fire
  Startup::XRE_Main

- 1.8584% (0.3240%):
  js::GCRuntime::collect
  nsJSContext::GarbageCollectNow
  Timer::Fire
  Startup::XRE_Main

- 1.7687% (0.9656%):
  ViewportFrame::BuildDisplayList
  nsLayoutUtils::GetFramesForArea
  nsLayoutUtils::GetFrameForPoint
  nsViewManager::DispatchEvent
  Timer::Fire
  Startup::XRE_Main

- 1.4348% (0.0003%):
  (content script)
  EventDispatcher::Dispatch
  Startup::XRE_Main

- 1.2465% (0.8883%):
  nsViewManager::DispatchEvent
  Timer::Fire
  Startup::XRE_Main

- 1.1803% (0.0500%):
  js::GCRuntime::collect
  nsRefreshDriver::Tick
  Startup::XRE_Main

- 0.9350% (0.0022%):
  (content script)
  (content script)
  Timer::Fire
  Startup::XRE_Main

- 0.6930% (0.1517%):
  nsCycleCollector::collectSlice
  nsJSContext